In [1]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from qiskit import *
from qiskit.circuit import Parameter
from qiskit import IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager

import numpy as np

import pickle

In [2]:
def scale_problems(problems, scale_min=-np.pi/4, scale_max=np.pi/4):
    scaled = []
    scaler = MinMaxScaler((-np.pi/4, np.pi/4))
    for entry in problems:
        scaled.append(np.array(scaler.fit_transform(entry.reshape(-1, 1)).flatten()))
    return scaled

In [3]:
def uncertainity_principle(circuit):
    circuit.h(range(circuit.width()))
    circuit.barrier()

def cost_encoding(circuit):
    for i in range(circuit.width()):
        circuit.ry(-Parameter('c'+str(i)), i)
    circuit.barrier()

def same_query_cost(circuit):
    for i in range(0,circuit.width(),2):
        circuit.crz(-np.pi/4,i,i+1)
    circuit.barrier()

def savings_encoding(circuit):
        for i in range(int(circuit.width()/2)):
            circuit.crz(Parameter('s'+str(i)+str(int(circuit.width()/2))), i, int(circuit.width()/2))
            circuit.crz(Parameter('s'+str(i)+str(int(1+circuit.width()/2))), i, int(1+circuit.width()/2))
        circuit.barrier()

def rx_layer(circuit, weight):
    circuit.rx(weight, range(circuit.width()))
    circuit.barrier()

def create_circuit(n_queries=2, n_plans=2, scheme="hcsx", xweight=np.pi/4):
    circuit = QuantumCircuit(n_queries*n_plans)
    for module in scheme:
        if module == "h":
            uncertainity_principle(circuit)
        elif module == "c":
            cost_encoding(circuit)
        elif module == "s":
            savings_encoding(circuit)
        elif module == "x":
            rx_layer(circuit, xweight)
    return circuit

In [4]:
def parse_results(result, max_value=True):
    remove_useless_keys(result)
    for i, key in enumerate(["0101", "1001", "0110", "1010"]):
        result[i] = result.pop(key, 0)
    if max_value:
        return max(result, key=result.get)
    else:
        return result

def remove_useless_keys(result):
    for key in ["0000","0001","0010","0100","1000","1101","1011","1100","0011","0111","1110","1111"]:
            if key in result:
                del result[key]
    for key in ["0101", "1001", "0110", "1010"]:
        if not key in result:
            result[key] = 0

def score_distance_results(results, solutions):
    x = np.array(results)
    y = np.array(solutions)
    distances = {}
    for r,s in zip(x,y):
        distance = np.where(s == r)[0]#[0]
        if len(distance) == 0:
            print('Solution not in prediction!')
            if not 'inf' in distances:
                distances['inf'] = 1
            else:
                distances['inf'] += 1
        elif not distance[0] in distances:
            distances[distance[0]] = 1
        else:
            distances[distance[0]] += 1
    return distances

In [5]:
def run_circuit(circuit, backend=Aer.get_backend("aer_simulator"), shots=1024):
    job = backend.run(transpile(circuit, backend), shots=shots)
    res = job.result()
    return res.get_counts(qc)

In [6]:
def score_results(results, solutions):
    return accuracy_score(results, solutions)

In [7]:
ibm_backend = 'ibmq_bogota'
total_iterations = 13

In [8]:
path = "data/problems_with_solutions.p"
data = pickle.load(open(path, "rb"))
x_train = scale_problems(data['x_train'])
x_test = scale_problems(data['x_test'])
y_train = data['y_train']
y_test = data['y_test']
y_test_ranking = data['y_test_ranked']

In [10]:
#load weights
weights_path = "experiments/optimizer_hyperparameters/amsgrad_final.p"
data = pickle.load(open(weights_path, "rb"))[0]

# ML OPTIMIZED SOLUTION

In [11]:
results = data['results'][0]
best_index = results['scoreTesting'].index(max(results['scoreTesting']))
rxweight = results['finalWeights'][best_index][0]
rxweight

0.23518365448276313

In [12]:
circuit = create_circuit(xweight=rxweight)

In [13]:
#prepare circuits
circuits = []
for test in x_test:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

### ML SIMULATION RUNS

In [14]:
data_to_save = {}

In [15]:
simulation_ml_accuracies = []
simulation_ml_parity_accuracies = []
simulation_ml_counts = []
for i in range(total_iterations):
    results_parity = []
    results_filtered = []
    c = []
    for qc in circuits:
        counts = run_circuit(qc.copy(), shots=4000)
        c.append(counts.copy())
        results_parity.append(int(max(counts, key=counts.get), 2) % 4)
        results_filtered.append(parse_results(counts))
    simulation_ml_accuracies.append(score_results(results_filtered, y_test))
    simulation_ml_parity_accuracies.append(score_results(results_parity, y_test))
    simulation_ml_counts.append(c)
data_to_save['ml_simulator_counts'] = simulation_ml_counts.copy() 

In [16]:
simulation_ml_filtered_counts = []
simulation_ml_parity_counts = []
for iteration in simulation_ml_counts:
    f = []
    p = []
    for counts in iteration:
        f.append(list({k: v for k, v in sorted(parse_results(counts.copy(), \
            max_value=False).items(), key=lambda item: item[1], reverse=True)}.keys()))
        parity_list = list({k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}.keys())[:4]
        p.append([int(a, 2)%4 for a in parity_list])
    simulation_ml_filtered_counts.append(f)
    simulation_ml_parity_counts.append(p)

In [17]:
simulation_ml_filtered_distances = []
simulation_ml_parity_distances = []
distances = []
for f, p in zip(simulation_ml_filtered_counts, simulation_ml_parity_counts):
    simulation_ml_filtered_distances.append(score_distance_results(f, y_test))
    simulation_ml_parity_distances.append(score_distance_results(p, y_test))

Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in pred

### ML REAL HARDWARE RUNS

In [18]:
provider = IBMQ.load_account()
backend = provider.get_backend(ibm_backend)
job_manager = IBMQJobManager()

In [19]:
transpiled_circuits = []
for qc in circuits:
    transpiled_circuits.append(transpile(qc, backend=backend))

In [20]:
real_hardware_counts = []
real_hardware_accuracies_parity = []
real_hardware_accuracies = []
counts = []
for i in range(total_iterations):
    print('Iteration ' + str(i+1))
    job_set_static = job_manager.run(transpiled_circuits, backend=backend, name='Static MQO ML Solver Iteration ' + str(i))
    real_results = job_set_static.results()
    results_parity = []
    results_filtered = []
    c = []
    for j in range(len(transpiled_circuits)):
        counts = real_results.get_counts(j)
        c.append(counts.copy())
        results_parity.append(int(max(counts, key=counts.get), 2) % 4)
        results_filtered.append(parse_results(counts.copy()))
    real_hardware_accuracies.append(score_results(results_filtered, y_test))
    real_hardware_accuracies_parity.append(score_results(results_parity, y_test))
    real_hardware_counts.append(c)
data_to_save['ml_real_hw_counts'] = real_hardware_counts.copy() 

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13


In [21]:
real_hw_ml_filtered_counts = []
real_hw_ml_parity_counts = []
for iteration in real_hardware_counts:
    f = []
    p = []
    for counts in iteration:
        f.append(list({k: v for k, v in sorted(parse_results(counts.copy(), \
            max_value=False).items(), key=lambda item: item[1], reverse=True)}.keys()))
        parity_list = list({k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}.keys())[:4]
        p.append([int(a, 2)%4 for a in parity_list])
    real_hw_ml_filtered_counts.append(f)
    real_hw_ml_parity_counts.append(p)

In [22]:
real_hardware_ml_filtered_distances = []
real_hardware_ml_parity_distances = []
distances = []
for f, p in zip(real_hw_ml_filtered_counts, real_hw_ml_parity_counts):
    real_hardware_ml_filtered_distances.append(score_distance_results(f, y_test))
    real_hardware_ml_parity_distances.append(score_distance_results(p, y_test))

Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in pred

In [252]:
real_hardware_ml_parity_distances

[{'inf': 18, 0: 23, 2: 8, 1: 22, 3: 4},
 {'inf': 18, 0: 24, 1: 22, 2: 7, 3: 4},
 {'inf': 18, 0: 23, 1: 21, 2: 9, 3: 4},
 {'inf': 18, 0: 24, 2: 10, 1: 20, 3: 3},
 {'inf': 18, 0: 26, 2: 9, 1: 19, 3: 3},
 {'inf': 18, 0: 24, 2: 12, 1: 18, 3: 3},
 {'inf': 18, 0: 23, 2: 11, 1: 19, 3: 4},
 {'inf': 19, 0: 24, 2: 9, 1: 20, 3: 3},
 {'inf': 18, 0: 24, 2: 10, 1: 19, 3: 4},
 {'inf': 18, 0: 23, 2: 9, 1: 20, 3: 5},
 {'inf': 17, 0: 22, 2: 13, 1: 19, 3: 4},
 {'inf': 18, 0: 22, 2: 12, 1: 20, 3: 3},
 {'inf': 17, 0: 23, 2: 10, 1: 21, 3: 4}]

# STATIC CIRCUIT RUNS

In [23]:
circuit = create_circuit()
#prepare circuits
circuits = []
for test in x_test:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

transpiled_circuits = []
for qc in circuits:
    transpiled_circuits.append(transpile(qc, backend=backend))

### SIMULATOR STATIC RUNS

In [24]:
simulation_accuracies = []
simulation_counts = []

for i in range(total_iterations):
    results = []
    distance_results = []
    c = []
    for qc in circuits:
        counts = run_circuit(qc.copy(), shots=4000)
        c.append(counts)
        results.append(parse_results(counts.copy()))
    simulation_accuracies.append(score_results(results, y_test))
    simulation_counts.append(c)
data_to_save['static_simulation_counts'] = simulation_counts.copy() 

In [25]:
simulation_distances = []
for f in simulation_counts:
        res = []
        for r in f:
                x = parse_results(r.copy(), max_value=False)
                x = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}.keys())
                res.append(x)
        simulation_distances.append(score_distance_results(res, y_test))

In [289]:
simulation_distances

[{0: 67, 1: 8},
 {0: 66, 2: 1, 1: 8},
 {0: 64, 1: 11},
 {0: 65, 1: 10},
 {0: 66, 1: 7, 2: 2},
 {0: 64, 1: 10, 2: 1},
 {0: 67, 2: 1, 1: 7},
 {0: 64, 1: 11},
 {0: 66, 2: 1, 1: 8},
 {0: 66, 1: 9},
 {0: 65, 2: 1, 1: 9},
 {0: 65, 1: 9, 2: 1},
 {0: 64, 1: 10, 2: 1}]

### REAL HARDWARE STATIC RUNS

In [26]:
real_accuracies = []
real_counts = []
for i in range(total_iterations):
    print('Iteration ' + str(i+1))
    job_set_static = job_manager.run(transpiled_circuits, backend=backend, name='Static MQO Solver Iteration ' + str(i))
    real_results = job_set_static.results()
    res_temp = []
    for j in range(len(transpiled_circuits)):
        real_counts.append(real_results.get_counts(j).copy())
        res_temp.append(parse_results(real_results.get_counts(j)))
    real_accuracies.append(score_results(res_temp, y_test))
data_to_save['static_real_hw_counts'] = real_counts.copy() 


Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13


In [27]:
real_hardware_distances = []
for f in real_counts:
        res = []
        for r in f:
                x = parse_results(r.copy(), max_value=False)
                x = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}.keys())
                res.append(x)
        real_hardware_distances.append(score_distance_results(res, y_test))

AttributeError: 'str' object has no attribute 'copy'

In [321]:
real_hardware_distances

[{1: 6, 0: 63, 2: 5, 3: 1},
 {1: 10, 0: 60, 2: 4, 3: 1},
 {1: 12, 0: 57, 2: 4, 3: 2},
 {1: 8, 0: 62, 2: 3, 3: 2},
 {1: 12, 0: 56, 3: 3, 2: 4},
 {1: 13, 0: 56, 2: 4, 3: 2},
 {0: 59, 1: 10, 2: 4, 3: 2},
 {1: 11, 0: 59, 2: 4, 3: 1},
 {1: 11, 0: 59, 2: 4, 3: 1},
 {0: 59, 1: 11, 3: 3, 2: 2},
 {1: 13, 0: 55, 2: 6, 3: 1},
 {1: 13, 0: 57, 2: 3, 3: 2},
 {1: 12, 0: 57, 2: 4, 3: 2}]

In [28]:
path = 'experiments/final_runs/'
savename = 'all_runs_done_accuracies.p'
try:
    os.makedirs(path)
except:
    print('path already exists!')

path already exists!


In [ ]:
run_data = {'static_simulator_filtered_accuracy':simulation_accuracies,
            'static_simulator_filtered_distances':simulation_distances, 
            'static_real_hw_filtered_accuracy': real_accuracies, 
            'static_real_hw_filtered_distances':real_hardware_distances,
            'ml_simulator_filtered_accuracy': simulation_ml_accuracies, 
            'ml_simulator_parity_accuracy': simulation_ml_parity_accuracies,
            'ml_simulator_filtered_distances': simulation_ml_filtered_distances, 
            'ml_simulator_parity_distances': simulation_ml_parity_distances,
            'ml_real_hw_filtered_accuracy': real_hardware_accuracies, 
            'ml_real_hw_parity_accuracy': real_hardware_accuracies_parity,
            'ml_real_hw_filtered_distances': real_hardware_ml_filtered_distances, 
            'ml_real_hw_parity_distances':real_hardware_ml_parity_distances,}

In [ ]:

pickle.dump(run_data, open( path+savename, "wb" ) )    

In [31]:
data_to_save.keys()

dict_keys(['ml_simulator_counts', 'ml_real_hw_counts', 'static_simulation_counts', 'static_real_hw_counts'])

In [29]:

pickle.dump(data_to_save, open( path+"counts_of_runs.p", "wb" ) )    